## **Import Required Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ee
import geemap

## **Set Up Earth Engine**

In [2]:
# Authenticating and initializing Earth Engine API
# ee.Authenticate()

In [3]:
# ee.Initialize()

In [4]:
# Creating a Map object
m = geemap.Map(width="100%", height="550px")

## **Add the Region of Interest**

In [5]:
# Define the region of interest
roi = ee.Geometry.Polygon([[88.282386, 22.519994], 
                           [88.391925, 22.519994],
                           [88.391925, 22.623027],
                           [88.282386, 22.623027],
                           [88.282386, 22.519994]])
# Display the roi
m.addLayer(roi, {}, "Region of Interest")
m.centerObject(roi, 12)
m

Map(center=[22.571513360067907, 88.33715550000028], controls=(WidgetControl(options=['position', 'transparent_…

## **Prepare the Data**

In [6]:
# Import Landsat 8 imagery
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

# Filter the image collection
l8Filtered = l8.filterBounds(roi)\
               .filterDate("2022-01-01", "2022-12-31")\
               .filterMetadata("CLOUD_COVER", "less_than", 10)\
               .select("SR_B.*")

# Create a median composite
composite = l8Filtered.median()\
                      .clip(roi)

In [7]:
# Create a function to apply scale factor
def applyScaleFactor(image):
    return image.multiply(0.0000275).add(-0.2)

# Apply the function over the composite image
scaled_composite = applyScaleFactor(composite)

In [8]:
# Display the selected image
rgbVis = {
    "min": 0.0,
    "max": 0.3,
    "bands": ["SR_B5", "SR_B4", "SR_B3"],
}
m.addLayer(scaled_composite, rgbVis, "Landsat 8 Composite Image")
m

Map(center=[22.571513360067907, 88.33715550000028], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
# Create a function to add Indices
def addIndices(image):
    ndvi = image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
    mndwi = image.normalizedDifference(["SR_B3", "SR_B6"]).rename("MNDWI")
    ndbi = image.normalizedDifference(["SR_B6", "SR_B5"]).rename("NDBI")
    savi = image.select("SR_B5").subtract(image.select("SR_B4"))\
                .divide(image.select("SR_B5").add(image.select("SR_B4").add(0.5)))\
                .multiply(1.5).rename("SAVI")
    
    return image.addBands(ndvi)\
                .addBands(mndwi)\
                .addBands(ndbi)\
                .addBands(savi)

# Apply the addIndices function over the scaled image
withIndices = addIndices(scaled_composite)
m.addLayer(withIndices, rgbVis, "Landsat 8 with Indices")
m

Map(center=[22.571513360067907, 88.33715550000028], controls=(WidgetControl(options=['position', 'transparent_…

## **Export the Imagery to Drive**

In [10]:
# Create a task to export the image
task = ee.batch.Export.image.toDrive(image=withIndices.toDouble(),
                                     description='Landsat_8_Image',
                                     folder="GEE",
                                     fileNamePrefix="Landsat_8_Image_Kolkata",
                                     region=roi,
                                     scale=30,
                                     maxPixels=1e10,
                                     fileFormat="GeoTIFF")

In [11]:
# Start the task
# task.start()